# Imports

In [1]:
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib as mpl
import itertools
import seaborn as sns
import cv2
import sklearn
from sklearn import datasets
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, auc
from functools import partial, lru_cache
from copy import copy
from scipy.special import softmax, expit

from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit, Aer, execute, IBMQ
from qiskit.tools.visualization import circuit_drawer
from qiskit.tools.visualization import plot_histogram
from qiskit.extensions.unitary import unitary
from qiskit.tools.monitor import job_monitor
from qiskit.compiler import transpile, assemble
S_simulator = Aer.backends(name = 'statevector_simulator')[0]
M_simulator = Aer.backends(name = 'qasm_simulator')[0]

import functools
from tqdm import tqdm, tqdm_notebook, notebook
from operator import itemgetter

%matplotlib qt
%load_ext jupyternotify

<IPython.core.display.Javascript object>

# Auxiliar functions definition

In [2]:
def preprocess(vector):
    """
    Input: a vector of size m = 2^N with entries in {-1, 1}.
    Output: a (m, 2) numpy array with the form:
    [[1 list([])]
     [-1 list([])]
     [1 list([])]
     [1 list([])]]
    where the frst column represents the sign and the second is the position in binary.
    """
    # Factorize the sign
    i_unsigned = np.multiply(-1, vector)
    # To binary string
    i_binary = [bin(int(n)) for n in i_unsigned]
    # Remove the -0b or 0b par of the binary representation. bin(7) = 0b111 -> 111
    for index, e in enumerate(i_binary):
        if(e[0] == '-'):
            #i_binary[index] = [-1, [int(e) for e in bin(index)[2:]]]
            i_binary[index] = [-1, list(map(int, str(int(bin(index)[2:]))))]
        else:
            #i_binary[index] = [1, [int(e) for e in bin(index)[2:]]]
            i_binary[index] = [1, list(map(int, str(int(bin(index)[2:]))))]
            
    n = int(np.log2(len(i_binary)))
    for index, element in enumerate(i_binary):
        if (len(element[1]) != n):
            while (len(i_binary[index][1]) != n):
                i_binary[index][1].insert(0, 0)
    #for i in range(len(i_binary)):
    #    i_binary[i][1] = i_binary[i][1][::-1]
    i_binary = np.array(i_binary)
    #print(i_binary)
    return i_binary

def check_z(binary):
    """
    Input: a preprocessed vector via preprocess().
    Output: a vector whose entries represents the qubit to perform a Z-gate.
    """
    positive = True # Controls the sign to achieve
    
    if (int(binary[0][0]) < 0):
        positive = False
        
    #print(f'The sign will be positive: {positive}')
    z_index = []
    for index, element in enumerate(binary):
        counts = Counter(element[1])
        #print(counts[1])
        if (counts[1] == 1):
            if (positive):
                if (int(binary[index][0]) == -1):
                    #print(f'Z gate in qubit {np.where(np.array(binary[2**i][1]) == 1)[0]}')
                    z_index.append(int(np.where(np.array(binary[index][1]) == 1)[0][0]))
            else:
                if (int(binary[index][0]) == 1):
                    #print(f'Z gate in qubit {np.where(np.array(binary[2**i][1]) == 1)[0]}')
                    z_index.append(int(np.where(np.array(binary[index][1]) == 1)[0][0]))
    #print(z_index)
    #z_index = reindex(int(np.log2(len(i_binary))), z_index)
    return z_index

def apply_z(binary, Z):
    """
    Input: the preprocessed vector and a vector containing the qubits to apply a Z-gate.
    Output: the resultant vector with the signs flipped accordingly to the action of the Z-gates.
    """
    '''for ind, e in enumerate(binary):
        counts = Counter(e[1])
        if counts[1] > 0:
            ones = np.where(np.array(e[1]) == 1)[0]
            #print(counts[1], e[1], ones)
            for z in Z:
                #print(f'z = {z}, ones = {ones}, changes = {z in ones}')
                if (z in ones):
                    binary[ind][0] *= -1'''
    
    for z in Z:
        mask = np.array(list(map(lambda x : x[z] == 1, binary[:,1])))
        binary[mask, 0] *= -1
    
    #print(binary)
    return binary


def check_cpz(binary):
    """
    Input: the current preprocessed vector.
    Output: the final desired vector and a vector whose elements are vectos containing the qubits to whom apply
            the CPZ.
    """
    CPZ = set()
    n = int(np.log2(len(binary)))
    #print(n)
    sequence = 0
    '''
    # Ordered by the number of ones in its binary representation
    aux_binary = sorted(binary, key = lambda item : Counter(item[1])[1])
    
    # Index holds the positions of the first appearance of n number of ones
    index = []
    cuenta = 1
    for i, e in enumerate(aux_binary):
        if (np.sum(e[1]) > cuenta):
            index.append(i)
            cuenta -= -1
    
    for ind in index: # Now ind holds where in the aux_binary star to appear elements with 2,3,... ones
        for i in range(ind, len(aux_binary)):
            if (aux_binary[i][0] != aux_binary[0][0]):
                ones = np.where(np.array(aux_binary[i][1]) == 1)[0]
                CPZ.add(tuple([sequence, tuple(ones)]))
                sequence -= -1
                
                for j in range(ind, len(aux_binary)):
                    pos = np.where(np.array(aux_binary[j][1]) == 1)[0]
                    pos_set = set(pos)
                    ones_set = set(ones)
                    if (set.issubset(ones_set, pos_set)):
                        aux_binary[j][0] *= -1
        
    
    
    
    '''
    for p in range(2, n + 1):
        for ind, e in enumerate(binary):
            counts = Counter(e[1])
            #print(f'manifold of {p}')
            if (counts[1] == p) and (binary[0][0] != e[0]):
                ones = np.where(np.array(e[1]) == 1)[0]
                #cadena = lambda a : str(a)
                #unos = cadena(ones)[1:-1]
                #CPZ.add(tuple(ones))
                CPZ.add(tuple([sequence, tuple(ones)]))
                #print(f'applying {tuple([sequence, tuple(ones)])} because {e}')
                sequence += 1
                
                ones_set = set(ones)
                mask = np.array(list(map(partial(check_set, ones_set) , binary[:, 1])))
                
                binary[mask, 0] *= -1
                
                #for ind, e in enumerate(binary):
                #    pos = np.where(np.array(e[1]) == 1)[0]
                #    pos_set = set(pos)
                #    ones_set = set(ones)
                #    if (set.issubset(ones_set, pos_set)):
                #        binary[ind][0] *= -1
                        #CPZ.append(ones)
                        #print(f'C{p}Z = {ones}')
    #'''
    #print(binary)
    CPZ = [list(ele) for _, ele in enumerate(CPZ)]
    CPZ = sorted(CPZ, key = lambda l: l[0])
    #print(CPZ)
    CPZ = [list(e[1]) for e in CPZ]
    #sett = [list(e) for e in sett]
    #if(binary[0][0] < 0):
    #    CPZ = list(map(partial(reindex, int(np.log2(len(binary)))), CPZ))
    #CPZ = sorted(CPZ, key = lambda l: (len(l), l.sort()))
    return CPZ

def check_set(ones_set, pos_set):
    return set.issubset(ones_set, set(np.where(np.array(pos_set) == 1)[0]))

def vectorize(number, digits):
    """
    Input: an integer in decimal form and the number of final digits as binary.
    Output: its representation on binary as an array. Most significant bit as first element of the array.
    """
    digits = 2**(np.ceil(np.log2(digits)))
    binary_number = bin(number)[2:]
    as_text = str(binary_number)
    vectorized_number = [int(d) for d in as_text]
    #print(vectorized_number)
    if (digits < len(vectorized_number)):
        digits = 2**(np.ceil(np.log2(len(vectorized_number))))
    if (digits > len(vectorized_number)):
        while (len(vectorized_number) != digits):
            vectorized_number.insert(0, 0)
    #print(vectorized_number)
    return np.array(vectorized_number)

def o_product(i, w):
    return (np.abs(np.dot(i, w))/len(i))**2

def sign(p):
    #Can be substitute in code by:
    #sign = lambda p : [(-1)**i for i in p]
    #list(map(sign, inputs))
    return [(-1)**i for i in p]

def design(p):
    return [1 if i < 0 else 0 for i in w]

def reindex(N, change):
    origin = np.array([i for i in range(N)])
    destiny = np.flipud(origin)
    
    for index, element in enumerate(change):
        #print(f'{element} -> {np.where(d == element)[0]}')
        change[index] = np.where(destiny == element)[0][0]
        
    return change

def U(qc, q, Z, CPZ):
    for z in Z:
        #print(f'Z gate in qubit {type(int(z))}')
        qc.z(q[int(z)])
    for cpz in CPZ:
        controls = [q[int(i)] for i in cpz[:-1]]
        target = q[int(cpz[-1])]
        #print(f'CPZ gate with control {controls} and target {target}')
        qc.h(cpz[-1])
        #qc.mct(controls, target, None, mode='advanced')
        
        #try:
        #    qc.mct(controls, target, None, mode='advanced')
        #except AssertionError:
            #print("Going to mode noancilla")
        qc.mct(controls, target, None, mode='noancilla')
        
        qc.h(cpz[-1])
    return qc

@lru_cache(maxsize = None)
def gates_sequence(vector):
    #print('Preprocessing')
    binary = preprocess(vector)
    #print('Getting Z gates')
    Z_GATES = check_z(binary)
    #print('Applying Z gates')
    binary = apply_z(binary, Z_GATES)
    #print('Getting CpZ gates')
    CPZ_GATES = check_cpz(binary)
    return Z_GATES, CPZ_GATES

def perceptron(N = 1, shots = 512, z_gates_w = [], CPZ_gates_w = [], z_gates_i = [], CPZ_gates_i = [], simulation = True):
    
    q = QuantumRegister(N, name = 'q_r')
    #A = QuantumRegister(1, name = 'auxiliar_ancillar')
    a = QuantumRegister(1, name = 'a_r')
    c = ClassicalRegister(1, name = 'c_r')
    qc = QuantumCircuit(q, a, c, name = 'qc')

    for index in range(N):
        qc.h(q[index])

        
    qc = U(qc, q, z_gates_i, CPZ_gates_i)
    qc = U(qc, q, z_gates_w, CPZ_gates_w)

    for index in range(N):
        qc.h(q[index])
    
    for index in range(N):
        qc.x(q[index])

    #qc.mct(q[:], a[0], None, mode='advanced')
    
    #try:
    #    qc.mct(q[:], a[0], None, mode='advanced')
    #except AssertionError:
        #print("Going to mode noancilla")
    qc.mct(q[:], a[0], None, mode='noancilla')    
        
    qc.measure(a[0], c)
    #print(qc)
        
    #results = execute(qc, backend = M_simulator, shots = shots).result()
    
    # Optimized
    transpiled_circuit = transpile(qc, M_simulator, optimization_level = 1) # optimization_level = [0, 1, 2, 3]
    job = M_simulator.run(assemble(transpiled_circuit, shots = shots))
    results = job.result()
    
    #results = execute(qc, backend = qcomp, shots = shots, memory=True) # For Real Quantum Computing

    #job_monitor(results) # For Real Quantum Computing
        
    #results = results.result() # For Real Quantum Computing
    
    # Optimized for Real Quantum Computer
    #transpiled_circuit = transpile(qc, qcomp, optimization_level = 2) # optimization_level = 0,1,2,3
    #job = M_simulator.run(assemble(transpiled_circuit, shots = shots))
    #results = job.result()
    
    return results


def fit(qubits = 1, X_train = np.array([[1]]), Y_train = np.array([[1]]), negative_class = 0, positive_class = 1, epochs = 1, shots = 64, threshold = 0.5, Ip = 0.5, In = 0.5, bias = 0):
    # Train and return a weight vector
    n = len(X_train)
    result_matrix = np.zeros((epochs, len(X_train)))
    np.random.seed(10)
    
    w = np.random.randint(2, size = 2**qubits) # Random generation of initial weight vector. Sign it at once.
    
    #w = np.array(X_train[23])
    
    #print(f'Starting training with qubits = {qubits}, train = {X_train}, target = {Y_train}, epochs = {epochs}, shots = {shots}, Ip = {Ip}, In = {In}')
    #print(f'Initial weight vector = {w}')
    
    weights = []

    for epoch in notebook.tqdm(range(epochs), desc = f'Epoches'):
        errors = 0
        correct = 0
        for index, training_element in enumerate(notebook.tqdm(X_train, desc = f'Training {epoch}')):

            pattern = sign(training_element) # Signing the training pattern.
            
            wei = sign(w) # Signing the weight vector
            #print('Calculating Z gates')
            #print(pattern)
            z_gates_i, CPZ_gates_i = gates_sequence(tuple(pattern)) # Gates to apply to the training pattern vector
            #print('Calculating CpZ gates')
            z_gates_w, CPZ_gates_w = gates_sequence(tuple(wei)) # Gates to apply to the weight vector
            
            #print('Ahora sí, Quantum Realm')
            
            ############################  Quantum Proccess  ############################
            
            results = perceptron(qubits, shots, z_gates_w, CPZ_gates_w, z_gates_i, CPZ_gates_i)
            
            #########################  End of Quantum Proccess  #########################
            
            counts = results.get_counts()
        
            #list_count_value = [ (v,oq.From_binary(k)) for k, v in counts.items()]

            #sorte = sorted(list_count_value, key = itemgetter(0))
        
            #readout = sorte[0][1] * sorte[0][0] / shots
            
            try:
                readout = counts['1'] / shots
            except KeyError:
                readout = 0
            
            #print(readout)
            
            clazz = negative_class if (readout + bias) < threshold else positive_class # Classification part.
            
            result_matrix[epoch][index] = readout + bias
            #print(f'Weight vector = {w} and Training element = {training_element}')
            if (Y_train[index] == clazz):
                # Correctly classified. We do not modify the weight vector and we continue.
                correct -= -np.power(np.inf, np.log(1))
                #if clazz:
                    #Ip *= (n - correct) / float(n)
                    #Ip *= 1.0 / np.log(3 + correct)
                    #Ip *= np.exp(-0.001 * correct)
                    #Ip *= 1.0 / (1.0 + 0.01 * correct)
                    #Ip *= .982**np.floor((correct + 1) / 10)
                #else:
                    #In *= (n - correct) / float(n)
                    #In *= 1.0 / np.log(3 + correct)
                    #In *= np.exp(-0.001 * correct)
                    #In *= 1.0 / (1.0 + 0.01 * correct)
                    #In *= .982**np.floor((correct + 1) / 10)
                continue # Skip the execution to the next iteration
            # If we reach this point the classification was incorrect
            #if (clazz < Y_train[index]): # Classified as 0 (negative) when is 1 (positive)
            if (clazz == negative_class): # Classified as (negative) when is (positive)
                # We flip +-1 signs Ip fraction where i and w coincide
                coincide = np.argwhere(training_element == w).flatten()
                pos = np.array(list(set(np.random.choice(coincide, int(np.floor(len(coincide) * Ip)), replace = False)))) if len(coincide) > 0 else np.array([])
                #print(f'Missclasiffied as 0. Before changes: {Counter(w)} with len(coincide) * Ip = {int(np.ceil(len(coincide) * Ip))}')
                #for indexed in pos:
                    #print('entre en Ip')
                #    w[indexed] = (w[indexed] + 1) % 2
#                print(f'Coincide in {len(coincide)} and change {len(pos)} bits')
                try:
                    w[pos] = (w[pos] + 1) % 2
                except IndexError:
                    pass
                #print(f'Missclasiffied as 0. After  changes: {Counter(w)} with len(coincide) * Ip = {int(np.ceil(len(coincide) * Ip))}')
            else: # Classified as 1 (positive) when is 0 (negative)
                # We flip +-1 signs In fraction where i and w coincide
                coincide = np.argwhere(training_element == w).flatten()
                pos = np.array(list(set(np.random.choice(coincide, int(np.floor(len(coincide) * In)), replace = False)))) if len(coincide) > 0 else np.array([])
                #print(f'Missclasified as 1. Before changes: {Counter(w)} with len(coincide) * In = {int(np.ceil(len(coincide) * In))}')
                #for indexed in pos:
                    #print('entre en In')
                #    w[indexed] = (w[indexed] + 1) % 2
#                print(f'Coincide in {len(coincide)} and change {len(pos)} bits')
                try:
                    w[pos] = (w[pos] + 1) % 2
                except IndexError:
                    pass
                #print(f'Missclasified as 1. After  changes: {Counter(w)} with len(coincide) * In = {int(np.ceil(len(coincide) * In))}')
            errors -= -1
        print(f'Errors = {errors} for accuracy of = {1 - errors/len(X_train)}')
        #print(f'Final weight vector = {w}')
        weights.append(w)
    return w, result_matrix, weights

def predict(qubits = 1, X_test = np.array([[1]]), negative_class = 0, positive_class = 1, shots = 64, threshold = 0.5, w = 0, bias = 0):
    # Train and return a weight vector
    result_matrix = np.zeros(len(X_test))
    readout_matrix = np.zeros(len(X_test))
    np.random.seed(10)
    Y_predicted = np.zeros(len(X_test))
    wei = sign(w) # Signing the weight vector
    z_gates_w, CPZ_gates_w = gates_sequence(tuple(wei)) # Gates to apply to the weight vector
    
    #print(f'Starting predict with qubits = {qubits}, train = {X_test}, shots = {shots}')
    #print(f'Passed weight vector = {w}')
         
    for index, training_element in enumerate(notebook.tqdm(X_test, desc = 'Predicting')):
            
        pattern = sign(training_element) # Signing the training pattern.
            
        z_gates_i, CPZ_gates_i = gates_sequence(tuple(pattern)) # Gates to apply to the training pattern vector
            
        ############################  Quantum Proccess  ############################
            
        results = perceptron(qubits, shots, z_gates_w, CPZ_gates_w, z_gates_i, CPZ_gates_i)
            
        #########################  End of Quantum Proccess  #########################
            
        counts = results.get_counts()
        
        #list_count_value = [ (v,oq.From_binary(k)) for k, v in counts.items()]

        #sorte = sorted(list_count_value, key = itemgetter(0))
        
        #readout = sorte[0][1] * sorte[0][0] / shots
        
        try:
            readout = counts['1'] / shots
        except KeyError:
            readout = 0
        
        #print(readout)
            
        clazz = negative_class if (readout + bias) < threshold else positive_class # Classification part.
            
        result_matrix[index] = clazz
        
        readout_matrix[index] = readout + bias

    return result_matrix, readout_matrix

def evaluate(Y_test, predicted, negative_class, positive_class):
    correct = np.argwhere(Y_test == predicted)
    accuracy = len(correct) / len(predicted)
    ones_and_zeros = [Y_test[ind][0] for ind in correct]
    counts = Counter(ones_and_zeros)
    confusion_matrix = np.zeros((2,2))
    
    Y_test_aux = np.zeros_like(Y_test)
    predicted_aux = np.zeros_like(predicted)
    
    #Y_test_aux[Y_test == negative_class] = 0
    #predicted_aux[predicted == negative_class] = 0
    Y_test_aux[Y_test == positive_class] = 1
    predicted_aux[predicted == positive_class] = 1
    
    for i in range(len(predicted_aux)):
        confusion_matrix[int(Y_test_aux[i])][int(predicted_aux[i])] += 1
    return accuracy, counts, confusion_matrix

def plot_confusion_matrix(cm, target_names = None, cmap = None, normalize = True, labels = True, title = 'Confusion Matrix'):
    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy
    
    if cmap is None:
        cmap = plt.get_cmap('Blues')
        
    if normalize:
        cm = cm.astype('float') / cm.sum(axis = 1)[:, np.newaxis]
        
    plt.figure(figsize = (8, 6))
    plt.imshow(cm, interpolation = 'nearest', cmap = cmap)
    plt.title(title)
    plt.colorbar()
    
    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    
    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names)
        plt.yticks(tick_marks, target_names)
        
    if labels:
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            if normalize:
                plt.text(j, i, "{:0.4f}".format(cm[i, j]), horizontalalignment = 'center', color = 'white' if cm[i, j] > thresh else 'black', fontsize = 20)
            else:
                plt.text(j, i, "{:,}".format(cm[i, j]), horizontalalignment = 'center', color = 'white' if cm[i, j] > thresh else 'black', fontsize = 20)
    
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy = {:0.4f}; misclass = {:0.4f}'.format(accuracy, misclass))
    plt.axis('equal')
    plt.show()
    
def save_weight(filename = 'weights.txt', mode = 'a+', weight = []):
    with open(filename, mode) as file:
        file.write(f'{number_negative_class, number_positive_class}:')
        for p in w:
            file.write(f'{p},')
        file.write('\n')
        
def retrieve_weights_from_file(filename = 'weights.txt', full = False):
    with open(filename, 'r+') as file:
        lines = file.readlines()
        
        cuadratic = lambda w : int(np.abs((1 - np.sqrt(1 + 8 * w)) / 2))
        
        nc = int(np.sqrt(len(lines))) if full else cuadratic(len(lines))
        
        weights_matrix = np.empty((nc, nc), dtype = object)
    
        #print(weights_matrix)
    
        for line in lines:
            #print(line, end = '\n')
            xy, peso = line.split(sep = ':')[0], line.split(sep = ':')[1]
            weight = np.array(list(map(int , peso.split(sep = ',')[:-1])))
            coordinates = [int(c) for c in xy if c.isdigit()]
            #print(f'Coordenates: {coordinates[0], coordinates[1]}, weight: {weight}')
            weights_matrix[coordinates[0]][coordinates[1]] = weight
            if not full:
                weights_matrix[coordinates[1]][coordinates[0]] = weight
    
        #print(weights_matrix)
    return weights_matrix

def retrieve_weights_from_file_total(filename = 'weights.txt'):
    with open(filename, 'r+') as file:
        lines = file.readlines()
        
        weights_vector = []
        
        for line in lines:
            xy, peso = line.split(sep = ':')[0], line.split(sep = ':')[1]
            weight = np.array(list(map(int , peso.split(sep = ',')[:-1])))
            weights_vector.append(weight)
            
        
    return np.array(weights_vector)

def print_weights_matrix(weights_matrix):
    n_classes = weights_matrix.shape[0]
    fig, axs = plt.subplots(2, n_classes // 2, figsize=(20, 10))
    count = 0

    for i in range(2):
        for j in range(n_classes // 2):
            ax = axs[i][j]
        
            #c = ax.pcolor(thetas[i + j][1:].reshape(28, 28), cmap = 'gray')
            try:
                c = ax.imshow(weights_matrix[i * 5 + j].reshape(8, 8), cmap = 'gray_r', interpolation='nearest')
                ax.axis('off')
                ax.axis('equal')
                ax.set_title(f'Weight vector for {i * 5 + j}').set_position([0.5, 0.9])
                count += 1
            except IndexError:
                continue
        
    fig.tight_layout()
    plt.colorbar(c)
    plt.show()
    
def multi_class_prediction_OVA(qubits = 1, classification_patterns = np.array([[1]]), shots = 64, test_threshold = 0.35, weights_matrix = np.zeros((1,1)), bias = 0):

    # Declaring and populating the Classification Matrix

    patterns = len(classification_patterns)
    holder_for_classes = np.zeros((patterns, 10), dtype = 'int64')
    holder_for_readouts = np.zeros((patterns, 10), dtype = 'float64')
    
    '''for i, x in enumerate(classification_patterns):
        for j, w in enumerate(weights_matrix):
            asigned_class, read_out = predict(qubits = Q, X_test = [x], negative_class = j, positive_class = 10, shots = shots, threshold = test_threshold, w = weights_matrix[j], bias = bias)
            holder_for_classes[i, j] = asigned_class
            holder_for_readouts[i, j] = read_out'''
            
    for i, w in enumerate(weights_matrix):
        asigned_class, read_out = predict(qubits = Q, X_test = classification_patterns, negative_class = i, positive_class = 10, shots = shots, threshold = test_threshold, w = weights_matrix[i], bias = bias)
        holder_for_classes[: ,i] = asigned_class
        holder_for_readouts[:, i] = read_out
    
    return holder_for_classes, holder_for_readouts

# Training binary imbalanced classifiers $sklearn.datasets.load\_digits()$

In [616]:
digits = datasets.load_digits()

In [617]:
# Getting images and labels from digits
images = digits.images
targets = digits.target

In [618]:
targets_count = Counter(targets)
print(f'Total count for digits: {targets_count}')

Total count for digits: Counter({3: 183, 1: 182, 5: 182, 4: 181, 6: 181, 9: 180, 7: 179, 0: 178, 2: 177, 8: 174})


In [619]:
images_and_labels = list(zip(digits.images, digits.target))
for index, (image, label) in enumerate(images_and_labels[45:65]):
    plt.subplot(2, 10, index + 1)
    plt.axis('off')
    plt.imshow(np.round(image / np.max(image)), cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title('Tr: %i' % label)

In [620]:
# Getting 0 and 1 from digits
number_negative_class = 9
number_positive_class = 10

In [621]:
targets[targets != number_negative_class] = number_positive_class

In [622]:
targets_count = Counter(targets)
print(f'Total count for digits: {targets_count}')

Total count for digits: Counter({10: 1617, 9: 180})


In [623]:
# NO SPLITTING FOR RESUSTITUTION ERROR MEASURE
X_train = copy(images)
X_test = copy(images)
Y_train = copy(targets)
Y_test = copy(targets)

In [624]:
images_and_labels = list(zip(X_test, Y_test))
for index, (image, label) in enumerate(images_and_labels[10:30]):
    plt.subplot(2, 10, index + 1)
    plt.axis('off')
    plt.imshow(image.reshape(8, 8), cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title('T: %i' % label)

In [625]:
# Reshaping digits
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

In [626]:
print(X_train.shape)
print(X_test.shape)

(1797, 64)
(1797, 64)


In [627]:
# Preprocessing digits by mapping in interval

for i in range(len(X_test)):
    X_test[i] = np.array(list(map(lambda x : 0 if 0 <= x < 10 else 1, X_test[i])))
    
for i in range(len(X_train)):
    X_train[i] = np.array(list(map(lambda x : 0 if 0 <= x < 10 else 1, X_train[i])))

In [628]:
'''X_train = X_train.tolist()
X_test = X_test.tolist()'''

'X_train = X_train.tolist()\nX_test = X_test.tolist()'

In [629]:
c = plt.imshow(np.array(X_train[35]).reshape(8, 8))
plt.colorbar(c)

## Fitting the classifier

In [631]:
Q = 6
test_threshold = .35
rs = 0
while rs < 0.80:
    w, m, weights = fit(qubits = Q, X_train = X_train[:], Y_train = Y_train[:], negative_class = number_negative_class, positive_class = number_positive_class, epochs = 1, shots = 1024, threshold = 0.35, Ip = .8, In = .1, bias = 0.0)
    classification, readouts = predict(qubits = Q, X_test = X_test, negative_class = number_negative_class, positive_class = number_positive_class, shots = 1024, threshold = test_threshold, w = weights[0], bias = 0.0)
    acc = accuracy_score(Y_test, classification)
    ps = precision_score(Y_test, classification, average = 'binary', pos_label = number_negative_class)
    rs = recall_score(Y_test, classification, average = 'binary', pos_label = number_negative_class)
    f1s = f1_score(Y_test, classification, average = 'binary', pos_label = number_negative_class)
    print(f'acc: {acc}, precision: {ps}, recall: {rs}, f1: {f1s}')


Errors = 1267 for accuracy of = 0.2949360044518642




acc: 0.36672231496939345, precision: 0.0905982905982906, recall: 0.5888888888888889, f1: 0.15703703703703703



Errors = 1272 for accuracy of = 0.29215358931552593




acc: 0.32109070673344464, precision: 0.10183767228177641, recall: 0.7388888888888889, f1: 0.17900403768506057



Errors = 1279 for accuracy of = 0.2882582081246522



...


acc: 0.25264329437952143, precision: 0.09135628952916373, recall: 0.7222222222222222, f1: 0.16219588271990018



Errors = 1266 for accuracy of = 0.2954924874791319




acc: 0.21702838063439064, precision: 0.07484407484407485, recall: 0.6, f1: 0.13308687615526804



Errors = 1243 for accuracy of = 0.3082915971062883




acc: 0.41958820255982193, precision: 0.13151152860802734, recall: 0.8555555555555555, f1: 0.22797927461139897


In [131]:
Q = 6
w, m, weights = fit(qubits = Q, X_train = X_train[:], Y_train = Y_train[:], negative_class = number_negative_class, positive_class = number_positive_class, epochs = 1, shots = 1024, threshold = 0.35, Ip = .8, In = .1, bias = 0.0)

Coincide in 34 and change 27 bits
Coincide in 20 and change 16 bits
Coincide in 19 and change 15 bits
Coincide in 15 and change 12 bits
Coincide in 16 and change 12 bits
Coincide in 14 and change 11 bits
Coincide in 23 and change 18 bits
Coincide in 20 and change 16 bits
Coincide in 15 and change 12 bits
Coincide in 15 and change 12 bits
Coincide in 19 and change 15 bits
Coincide in 17 and change 13 bits
Coincide in 20 and change 16 bits
Coincide in 19 and change 15 bits
Coincide in 20 and change 16 bits
Coincide in 19 and change 15 bits
Coincide in 17 and change 13 bits
Coincide in 18 and change 14 bits
Coincide in 19 and change 15 bits
Coincide in 15 and change 12 bits
Coincide in 19 and change 15 bits
Coincide in 18 and change 14 bits
Coincide in 16 and change 12 bits
Coincide in 17 and change 13 bits
Coincide in 17 and change 13 bits
Coincide in 17 and change 13 bits
Coincide in 18 and change 14 bits
Coincide in 17 and change 13 bits
Coincide in 20 and change 16 bits
Coincide in 15

Coincide in 17 and change 13 bits
Coincide in 16 and change 12 bits
Coincide in 16 and change 12 bits
Coincide in 16 and change 12 bits
Coincide in 18 and change 14 bits
Coincide in 20 and change 16 bits
Coincide in 19 and change 15 bits
Coincide in 17 and change 13 bits
Coincide in 21 and change 16 bits
Coincide in 16 and change 12 bits
Coincide in 19 and change 15 bits
Coincide in 17 and change 13 bits
Coincide in 14 and change 11 bits
Coincide in 18 and change 14 bits
Coincide in 14 and change 11 bits
Coincide in 16 and change 12 bits
Coincide in 18 and change 14 bits
Coincide in 20 and change 16 bits
Coincide in 23 and change 18 bits
Coincide in 19 and change 15 bits
Coincide in 15 and change 12 bits
Coincide in 17 and change 13 bits
Coincide in 14 and change 11 bits
Coincide in 9 and change 0 bits
Coincide in 13 and change 10 bits
Coincide in 16 and change 12 bits
Coincide in 16 and change 12 bits
Coincide in 14 and change 11 bits
Coincide in 15 and change 12 bits
Coincide in 18 a

Coincide in 13 and change 10 bits
Coincide in 19 and change 15 bits
Coincide in 16 and change 12 bits
Coincide in 17 and change 13 bits
Coincide in 16 and change 12 bits
Coincide in 20 and change 16 bits
Coincide in 18 and change 14 bits
Coincide in 15 and change 12 bits
Coincide in 9 and change 0 bits
Coincide in 9 and change 0 bits
Coincide in 13 and change 10 bits
Coincide in 19 and change 15 bits
Coincide in 18 and change 14 bits
Coincide in 15 and change 12 bits
Coincide in 26 and change 20 bits
Coincide in 20 and change 16 bits
Coincide in 18 and change 14 bits
Coincide in 23 and change 18 bits
Coincide in 23 and change 18 bits
Coincide in 18 and change 14 bits
Coincide in 16 and change 12 bits
Coincide in 20 and change 16 bits
Coincide in 16 and change 12 bits
Coincide in 19 and change 15 bits
Coincide in 18 and change 14 bits
Coincide in 16 and change 12 bits
Coincide in 17 and change 13 bits
Coincide in 12 and change 1 bits
Coincide in 17 and change 13 bits
Coincide in 18 and 

Coincide in 23 and change 18 bits
Coincide in 21 and change 16 bits
Coincide in 15 and change 12 bits
Coincide in 19 and change 15 bits
Coincide in 16 and change 12 bits
Coincide in 19 and change 15 bits
Coincide in 17 and change 13 bits
Coincide in 14 and change 11 bits
Coincide in 16 and change 12 bits
Coincide in 15 and change 12 bits
Coincide in 19 and change 15 bits
Coincide in 21 and change 16 bits
Coincide in 17 and change 13 bits
Coincide in 17 and change 13 bits
Coincide in 16 and change 12 bits
Coincide in 14 and change 11 bits
Coincide in 8 and change 0 bits
Coincide in 15 and change 12 bits
Coincide in 16 and change 12 bits
Coincide in 22 and change 17 bits
Coincide in 16 and change 12 bits
Coincide in 18 and change 14 bits
Coincide in 15 and change 12 bits
Coincide in 14 and change 11 bits
Coincide in 17 and change 13 bits
Coincide in 17 and change 13 bits
Coincide in 18 and change 14 bits
Coincide in 16 and change 12 bits
Coincide in 14 and change 11 bits
Coincide in 16 a

Coincide in 20 and change 16 bits
Coincide in 20 and change 16 bits
Coincide in 17 and change 13 bits
Coincide in 20 and change 16 bits
Coincide in 13 and change 1 bits
Coincide in 22 and change 17 bits
Coincide in 19 and change 15 bits
Coincide in 14 and change 11 bits
Coincide in 18 and change 14 bits
Coincide in 16 and change 12 bits
Coincide in 17 and change 13 bits
Coincide in 18 and change 14 bits
Coincide in 22 and change 17 bits
Coincide in 19 and change 15 bits
Coincide in 15 and change 12 bits
Coincide in 17 and change 13 bits
Coincide in 17 and change 13 bits
Coincide in 19 and change 15 bits
Coincide in 17 and change 13 bits
Coincide in 13 and change 10 bits
Coincide in 16 and change 12 bits
Coincide in 24 and change 19 bits
Coincide in 19 and change 15 bits
Coincide in 18 and change 14 bits
Coincide in 20 and change 16 bits
Coincide in 18 and change 14 bits
Coincide in 12 and change 1 bits
Coincide in 18 and change 14 bits
Coincide in 24 and change 19 bits
Coincide in 19 a

Coincide in 17 and change 13 bits
Coincide in 19 and change 15 bits
Coincide in 13 and change 10 bits
Coincide in 15 and change 12 bits
Coincide in 15 and change 12 bits
Coincide in 15 and change 12 bits
Coincide in 16 and change 12 bits
Coincide in 14 and change 11 bits
Coincide in 14 and change 11 bits
Coincide in 16 and change 12 bits
Coincide in 16 and change 12 bits
Coincide in 16 and change 12 bits
Coincide in 18 and change 14 bits
Coincide in 18 and change 14 bits
Coincide in 17 and change 13 bits
Coincide in 12 and change 1 bits
Coincide in 18 and change 14 bits
Coincide in 13 and change 10 bits
Coincide in 15 and change 12 bits
Coincide in 15 and change 12 bits
Coincide in 14 and change 11 bits
Coincide in 17 and change 13 bits
Coincide in 15 and change 12 bits
Coincide in 23 and change 18 bits
Coincide in 15 and change 12 bits
Coincide in 14 and change 11 bits
Coincide in 20 and change 16 bits
Coincide in 17 and change 13 bits
Coincide in 19 and change 15 bits
Coincide in 14 

In [632]:
c = plt.imshow(np.array(w).reshape(8, 8), cmap = 'gray_r')
plt.colorbar(c)

## Predicting and evaluating the trained classifier

In [635]:
test_threshold = .35
classification, readouts = predict(qubits = Q, X_test = X_test, negative_class = number_negative_class, positive_class = number_positive_class, shots = 1024, threshold = test_threshold, w = weights[0], bias = 0.0)

In [636]:
accuracy, counts, confusion_matriX = evaluate(Y_test, classification, number_negative_class, number_positive_class)
print(f'Accuracy = {accuracy}')
print(f'Counts = {counts}')
print(f'Confusion Matrix = \n{confusion_matriX}')

Accuracy = 0.41958820255982193
Counts = Counter({10: 599, 9: 155})
Confusion Matrix = 
[[ 155.   25.]
 [1018.  599.]]


In [637]:
plot_confusion_matrix(confusion_matriX, target_names = np.array([number_negative_class, number_positive_class]), labels = True, normalize = False)

In [638]:
confusion_matriX

array([[ 155.,   25.],
       [1018.,  599.]])

In [639]:
accuracy_score(Y_test, classification)

0.41958820255982193

In [640]:
confusion_matrix(Y_test, classification)

array([[ 155,   25],
       [1018,  599]], dtype=int64)

In [641]:
precision_score(Y_test, classification, average = 'binary', pos_label = number_negative_class)

0.13213981244671782

In [642]:
recall_score(Y_test, classification, average = 'binary', pos_label = number_negative_class)

0.8611111111111112

In [643]:
f1_score(Y_test, classification, average = 'binary', pos_label = number_negative_class)

0.22912047302291208

In [644]:
%%notify -o
f'The accuracy is {accuracy}.'

'The accuracy is 0.41958820255982193.'

<IPython.core.display.Javascript object>

In [645]:
test = [0 if ele == number_negative_class else 1 for ele in Y_test]
classi = [0 if ele == number_negative_class else 1 for ele in classification]

fpr, tpr, thresholds = roc_curve(test, classi)

def ROC_curve(fpr,tpr):
    # Seaborn's beautiful styling
    sns.set_style('darkgrid', {'axes.facecolor': '0.9'})
    print('AUC: {}'.format(auc(fpr, tpr)))
    plt.figure(figsize=(10, 8))
    lw = 2
    plt.plot(fpr, tpr, color='darkorange',
             lw=lw, label='ROC curve')
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.yticks([i/20.0 for i in range(21)], fontsize = 15)
    plt.xticks([i/20.0 for i in range(21)], fontsize = 15)
    plt.xlabel('False Positive Rate', fontsize = 20)
    plt.ylabel('True Positive Rate', fontsize = 20)
    plt.title('Receiver operating characteristic (ROC) Curve', fontsize = 20)
    plt.legend(loc='lower right')
    plt.show()

ROC_curve(fpr,tpr)
sns.set_style('white', {'axes.facecolor': '0.9'})


AUC: 0.615775097917955


## Manual process analysis

In [646]:
classification.astype(int)

array([10, 10, 10, ..., 10,  9, 10])

In [647]:
Y_test

array([10, 10, 10, ..., 10,  9, 10])

In [648]:
fails = np.where(classification != Y_test)[0]
print(fails)

[   4    5    7 ... 1788 1791 1792]


In [649]:
X_test = np.array(X_test)
for x in X_test[fails]:
    print(np.power((np.where(weights[0] == x)[0].shape[0] - np.where(weights[0] != x)[0].shape[0]) / x.shape[0], 2))

0.3525390625
0.31640625
0.09765625
0.25
0.25
0.2822265625
0.31640625
0.25
0.2822265625
0.31640625
0.31640625
0.25
0.25
0.25
0.19140625
0.2197265625
0.25
0.2822265625
0.25
0.31640625
0.25
0.31640625
0.25
0.1650390625
0.2197265625
0.1650390625
0.2822265625
0.25
0.31640625
0.2822265625
0.1650390625
0.2822265625
0.2197265625
0.3525390625
0.31640625
0.1650390625
0.25
0.3525390625
0.2822265625
0.25
0.31640625
0.1650390625
0.390625
0.2822265625
0.25
0.2197265625
0.1650390625
0.2822265625
0.25
0.2822265625
0.2197265625
0.3525390625
0.2197265625
0.19140625
0.19140625
0.3525390625
0.3525390625
0.31640625
0.31640625
0.25
0.2822265625
0.19140625
0.140625
0.25
0.2822265625
0.2822265625
0.2197265625
0.31640625
0.31640625
0.2822265625
0.2822265625
0.3525390625
0.31640625
0.2822265625
0.390625
0.0791015625
0.140625
0.2197265625
0.2197265625
0.2822265625
0.140625
0.31640625
0.25
0.31640625
0.31640625
0.3525390625
0.31640625
0.0791015625
0.2822265625
0.25
0.1650390625
0.25
0.140625
0.3525390625
0.25
0.2

In [650]:
print(np.sort(readouts[np.where(Y_test == number_negative_class)]), end = '\n\n')
print(np.sort(readouts[np.where(Y_test == number_positive_class)]))
print(np.sort(readouts[fails]))

[0.05859375 0.0625     0.06738281 0.06738281 0.06933594 0.0703125
 0.078125   0.07910156 0.08496094 0.08691406 0.08886719 0.08984375
 0.08984375 0.09570312 0.10742188 0.10742188 0.10839844 0.11230469
 0.11523438 0.12109375 0.12304688 0.12304688 0.12695312 0.13183594
 0.13183594 0.13476562 0.14257812 0.14941406 0.15234375 0.15332031
 0.15820312 0.15917969 0.16015625 0.16113281 0.16308594 0.16308594
 0.16503906 0.16503906 0.171875   0.17285156 0.17382812 0.17480469
 0.17675781 0.17773438 0.17773438 0.1796875  0.1796875  0.18164062
 0.18164062 0.18261719 0.18359375 0.18359375 0.18554688 0.18652344
 0.18652344 0.18652344 0.1875     0.18847656 0.19042969 0.19238281
 0.1953125  0.19921875 0.19921875 0.20117188 0.20507812 0.20507812
 0.20605469 0.20996094 0.2109375  0.2109375  0.21484375 0.21777344
 0.21777344 0.21972656 0.22167969 0.22363281 0.22460938 0.2265625
 0.22753906 0.22949219 0.23242188 0.24023438 0.24609375 0.24707031
 0.24707031 0.24804688 0.25195312 0.25292969 0.25390625 0.254882

In [305]:
fig, axs = plt.subplots(3, int(np.ceil(len(fails) / 3)), figsize=(15,9))
count = 0

for i in range(3):
    for j in range(int(np.ceil(len(fails) / 3))):
        ax = axs[i][j]
        
        #c = ax.pcolor(thetas[i + j][1:].reshape(28, 28), cmap = 'gray')
        try:
            c = ax.imshow(np.array(X_test[fails[count]]).reshape(8, 8), cmap = 'gray_r', interpolation='nearest')
            ax.axis('off')
            ax.axis('equal')
            ax.set_title(f'Classified as {classification[fails[count]]}').set_position([0.5, 1.1])
            count += 1
        except IndexError:
            continue
        
fig.tight_layout()
plt.colorbar(c)
plt.show()

KeyboardInterrupt: 

In [651]:
Counter(Y_test)

Counter({10: 1617, 9: 180})

In [652]:
readouts

array([0.42871094, 0.45605469, 0.42089844, ..., 0.80957031, 0.32519531,
       0.47167969])

In [653]:
indexes = [i for i in range(len(readouts))]

In [654]:
list(zip(indexes, Y_test, readouts))

[(0, 10, 0.4287109375),
 (1, 10, 0.4560546875),
 (2, 10, 0.4208984375),
 (3, 10, 0.4072265625),
 (4, 10, 0.3408203125),
 (5, 10, 0.3232421875),
 (6, 10, 0.5),
 (7, 10, 0.103515625),
 (8, 10, 0.4912109375),
 (9, 9, 0.1865234375),
 (10, 10, 0.3642578125),
 (11, 10, 0.2548828125),
 (12, 10, 0.2568359375),
 (13, 10, 0.2763671875),
 (14, 10, 0.4365234375),
 (15, 10, 0.326171875),
 (16, 10, 0.244140625),
 (17, 10, 0.28515625),
 (18, 10, 0.3125),
 (19, 9, 0.1318359375),
 (20, 10, 0.291015625),
 (21, 10, 0.2265625),
 (22, 10, 0.3603515625),
 (23, 10, 0.2490234375),
 (24, 10, 0.2509765625),
 (25, 10, 0.1865234375),
 (26, 10, 0.5498046875),
 (27, 10, 0.2099609375),
 (28, 10, 0.482421875),
 (29, 9, 0.2197265625),
 (30, 10, 0.2529296875),
 (31, 9, 0.123046875),
 (32, 10, 0.291015625),
 (33, 10, 0.267578125),
 (34, 10, 0.37109375),
 (35, 10, 0.3125),
 (36, 10, 0.4228515625),
 (37, 9, 0.1123046875),
 (38, 10, 0.2333984375),
 (39, 9, 0.2724609375),
 (40, 10, 0.4716796875),
 (41, 10, 0.3271484375),
 (

In [655]:
#%matplotlib qt
plt.scatter(Y_train, m[-1])
plt.xlabel("Number")
plt.ylabel("Training Readout")
#%matplotlib inline

Text(0, 0.5, 'Training Readout')

In [656]:
#%matplotlib qt
plt.scatter(Y_test, readouts)
plt.xlabel("Number")
plt.ylabel("Prediction Readout")
plt.plot([i for i in range(number_negative_class, number_positive_class + 1)], [test_threshold for _ in range(number_negative_class, number_positive_class + 1)])
plt.plot([i for i in range(min(number_negative_class, number_positive_class), max(number_negative_class, number_positive_class) + 1)], [test_threshold for _ in range(abs(number_negative_class - number_positive_class) + 1)])
#%matplotlib inline

In [657]:
classification_auxiliar = classification / np.max(classification)
palette = np.array(sns.color_palette("hls", 2))  #Choosing color palette 
plt.scatter(Y_test, readouts, lw=0, s=40, c=palette[classification_auxiliar.astype(np.int)])
plt.plot([i for i in range(number_negative_class, number_positive_class + 1)], [test_threshold for _ in range(number_negative_class, number_positive_class + 1)])

In [658]:
Y_test

array([10, 10, 10, ..., 10,  9, 10])

In [659]:
plt.imshow(np.array(X_test[2]).reshape(8,8), cmap=plt.cm.gray_r, interpolation='nearest')

In [660]:
np.array(X_test[2]).reshape(8,8)

array([[0., 0., 0., 0., 1., 1., 0., 0.],
       [0., 0., 0., 1., 1., 1., 0., 0.],
       [0., 0., 0., 1., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 1., 0., 0.],
       [0., 0., 0., 1., 1., 0., 0., 0.],
       [0., 0., 1., 1., 0., 0., 0., 0.],
       [0., 0., 1., 1., 1., 1., 0., 0.],
       [0., 0., 0., 0., 1., 1., 0., 0.]])

In [661]:
import seaborn as sns
sns.set_style('white')
sns.distplot(readouts, bins = 100, kde = False)

In [662]:
palette = np.array(sns.color_palette("hls", 2))  #Choosing color palette 
plt.scatter(Y_train, m[-1], lw=0, s=40, c=palette[m[-1].astype(np.int)])
plt.plot([i for i in range(number_negative_class, number_positive_class + 1)], [test_threshold for _ in range(number_negative_class, number_positive_class + 1)])

In [663]:
filename = 'weights_OvA.txt'
mode = 'a+'
save_weight(filename = filename, mode = mode, weight = w)

# Multi label prediction

In [3]:
weights_vector = retrieve_weights_from_file_total(filename = 'weights_OvA.txt')

In [4]:
print_weights_matrix(weights_vector)

In [5]:
print(weights_vector)

[[1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 0 0 0 1 1 1 1 1 0 0 1 1 1 1 1 1 0
  0 0 1 1 1 1 1 0 0 0 1 1 1 1 1 0 0 0 1 1 1 1 1 1 0 1 1 1]
 [1 1 1 1 0 1 1 1 1 1 0 0 0 0 1 1 1 1 0 1 1 0 1 1 1 1 0 0 0 0 1 1 1 1 1 1
  0 0 1 1 1 1 0 0 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0 0 1 1 1]
 [1 1 1 0 1 1 1 1 1 1 0 0 0 0 1 1 1 1 0 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 0
  0 0 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 0 1 1 1 1 1 1 0 0 1 1]
 [1 1 1 0 1 1 1 1 1 1 0 0 0 1 1 1 1 1 0 1 1 0 1 1 1 1 0 0 1 0 1 1 1 1 1 0
  0 0 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1]
 [1 1 1 0 1 1 1 1 1 1 0 0 0 0 1 1 1 1 0 1 1 0 1 1 1 1 0 0 1 0 1 1 1 1 1 0
  1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 0 1 1 1 1 1 0 0 1 1 1]
 [1 1 1 0 1 1 1 1 1 1 0 0 1 0 1 1 1 1 0 0 1 0 1 1 1 1 1 0 0 0 1 1 1 1 0 0
  0 0 1 1 1 1 0 1 1 0 1 1 1 1 0 0 1 0 1 1 1 1 1 0 0 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 0 0 0 0 1 1 1 1 0 1 1 0 1 1 1 1 0 0 1 0 1 1 1 1 1 0
  0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0 0 1 1 1]
 [1 1 1 0 0 1 1 1 1 1 0 0 0 1 1 1 1 1 0 1 1 0 1 1 1 1 0 0 1 0 1 1 1 1

In [6]:
digits = datasets.load_digits()

# Getting images and labels from digits
images = digits.images
targets = digits.target

print(f'Total count for digits: {Counter(targets)}')
# Splitting dataset
#X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(images, targets, test_size = 0.2, random_state = 10, shuffle = True)
X_test, Y_test = images, targets

Total count for digits: Counter({3: 183, 1: 182, 5: 182, 4: 181, 6: 181, 9: 180, 7: 179, 0: 178, 2: 177, 8: 174})


In [7]:
images_and_labels = list(zip(X_test, Y_test))
for index, (image, label) in enumerate(images_and_labels[:20]):
    plt.subplot(2, 10, index + 1)
    plt.axis('off')
    plt.imshow(image.reshape(8, 8), cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title('Test: %i' % label)

In [8]:
# Reshaping digits
#X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)


# Preprocessing digits by mapping in interval
for i in range(len(X_test)):
    X_test[i] = np.array(list(map(lambda x : 0 if 0 <= x < 9 else 1, X_test[i])))
    
#for i in range(len(X_train)):
#    X_train[i] = np.array(list(map(lambda x : 0 if 0 <= x < 9 else 1, X_train[i])))

In [9]:
images_and_labels = list(zip(X_test, Y_test))
for index, (image, label) in enumerate(images_and_labels[:20]):
    plt.subplot(2, 10, index + 1)
    plt.axis('off')
    plt.imshow(image.reshape(8, 8), cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title('T: %i' % label)

C:\Users\jeff_\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
N = -1
test_threshold = .35
Q = 6
classification_patterns = X_test[:N]
classification_labels = Y_test[:N]
shots = 1024
bias = 0.0

holder_for_classes, holder_for_readouts = multi_class_prediction_OVA(qubits = Q, classification_patterns = classification_patterns, shots = shots, test_threshold = test_threshold, weights_matrix = weights_vector, bias = bias)

In [11]:
# To print only two decimal places
np.set_printoptions(precision = 2)

In [12]:
holder_for_classes

array([[ 0, 10, 10, ..., 10, 10, 10],
       [10,  1,  2, ...,  7,  8, 10],
       [ 0,  1,  2, ...,  7,  8, 10],
       ...,
       [ 0, 10,  2, ...,  7, 10, 10],
       [10,  1,  2, ...,  7,  8, 10],
       [ 0, 10, 10, ..., 10, 10,  9]], dtype=int64)

In [13]:
mx, my = np.where(holder_for_classes == 10)

In [14]:
for i, j in zip(mx, my):
    holder_for_readouts[i, j] = 0

In [15]:
print(holder_for_readouts)

[[0.23 0.   0.   ... 0.   0.   0.  ]
 [0.   0.24 0.23 ... 0.28 0.16 0.  ]
 [0.18 0.25 0.16 ... 0.21 0.16 0.  ]
 ...
 [0.23 0.   0.32 ... 0.33 0.   0.  ]
 [0.   0.34 0.25 ... 0.29 0.2  0.  ]
 [0.33 0.   0.   ... 0.   0.   0.3 ]]


In [25]:
classified_as = np.argmax(holder_for_readouts, axis = 1)

In [82]:
fig, axs = plt.subplots(3, int(np.ceil(len(classification_patterns) / 3)), figsize=(15,9))
count = 0

for i in range(3):
    for j in range(int(np.ceil(len(classification_patterns) / 3))):
        ax = axs[i][j]
        
        #c = ax.pcolor(thetas[i + j][1:].reshape(28, 28), cmap = 'gray')
        try:
            c = ax.imshow(np.array(classification_patterns[count]).reshape(8, 8), cmap = 'gray_r', interpolation='nearest')
            ax.axis('off')
            ax.axis('equal')
            ax.set_title(f'Clas. as {classified_as[count]} is {classification_labels[count]}').set_position([0.5, 1.1])
            count += 1
        except IndexError:
            continue
        
fig.tight_layout()
plt.colorbar(c)
plt.show()

In [26]:
accuracy = len(np.where(classification_labels == classified_as)[0]) / len(classification_labels)
print(f'Accuracy = {accuracy}')

Accuracy = 0.09855233853006681


In [18]:
fig, ax = plt.subplots(1, 1, figsize=(15,9))

c = ax.imshow(holder_for_readouts, cmap = 'hot', interpolation='nearest')
ax.axis('on')
#ax.axis('equal')
ax.set_title(f'Heatmap for Readouts').set_position([0.5, 1.1])
        
fig.tight_layout()
norm = mpl.colors.Normalize(vmin=np.nan_to_num(holder_for_readouts.astype(float)).min(), vmax=np.nan_to_num(holder_for_readouts.astype(float)).max())
plt.colorbar(c, norm = norm)
plt.show()

## Evaluating

In [27]:
conf = confusion_matrix(classification_labels, classified_as)
conf

array([[43, 12, 14, 18, 13, 10, 11, 17, 10, 30],
       [22, 24, 11, 11,  6, 25,  9, 50,  9, 15],
       [38, 13,  5,  6, 12, 16,  2, 24, 11, 50],
       [24, 25, 15, 11, 16, 11, 18, 10, 36, 17],
       [26, 35,  1, 12,  5, 19, 13, 18, 20, 32],
       [18, 16, 18, 14, 25,  9, 10, 24, 13, 35],
       [14, 12, 10, 31, 18, 20, 16, 28,  8, 24],
       [17, 25,  4,  3, 22, 29, 17, 25, 16, 21],
       [50, 11, 12, 10, 17, 14, 17, 16, 11, 15],
       [27, 19, 11, 20, 10, 21, 13, 15, 16, 28]], dtype=int64)

In [28]:
plot_confusion_matrix(conf, target_names = np.unique(classification_labels), labels = True, normalize = False)

In [29]:
accuracy_score(classification_labels, classified_as)

0.09855233853006681

In [30]:
confusion_matrix(classification_labels, classified_as)

array([[43, 12, 14, 18, 13, 10, 11, 17, 10, 30],
       [22, 24, 11, 11,  6, 25,  9, 50,  9, 15],
       [38, 13,  5,  6, 12, 16,  2, 24, 11, 50],
       [24, 25, 15, 11, 16, 11, 18, 10, 36, 17],
       [26, 35,  1, 12,  5, 19, 13, 18, 20, 32],
       [18, 16, 18, 14, 25,  9, 10, 24, 13, 35],
       [14, 12, 10, 31, 18, 20, 16, 28,  8, 24],
       [17, 25,  4,  3, 22, 29, 17, 25, 16, 21],
       [50, 11, 12, 10, 17, 14, 17, 16, 11, 15],
       [27, 19, 11, 20, 10, 21, 13, 15, 16, 28]], dtype=int64)

In [31]:
precision_score(classification_labels, classified_as, average = None)

array([0.15, 0.12, 0.05, 0.08, 0.03, 0.05, 0.13, 0.11, 0.07, 0.1 ])

In [32]:
recall_score(classification_labels, classified_as, average = None)

array([0.24, 0.13, 0.03, 0.06, 0.03, 0.05, 0.09, 0.14, 0.06, 0.16])

In [33]:
f1_score(classification_labels, classified_as, average = None)

array([0.19, 0.13, 0.04, 0.07, 0.03, 0.05, 0.1 , 0.12, 0.07, 0.13])